# Indexing in torch

## common indexing

利用整数坐标 $index$，或者切片 $start:end:step$ 进行索引。两者可组合使用。

使用整数坐标时，该维度将会消失（取出元素动作）。

使用切片时，该维度会被保留（切取片段）。

## bool indexing

通过 *bool tensor* 作为索引，索引至位置为 True 的位置，可进行赋值和取值。

在 bool indexing 和前述 common indexing 混用时，优先处理完 common indexing，剩余维度继续匹配 bool indexing。

当 *bool tensor* 为 1-dim 时，其大小须等于被索引维度的大小，下标对应 True 的位置被索引到。

当 *bool tensor* 为 n-dim 时，匹配剩余维度之后（严格每一维大小相等），将其余维度视作整体元素，剩余维度之中下标对应 True 的位置被索引到。(建议不要在 *bool indexing* 之后的维度用 *common indexing*，要使用也请选择先 transpose 交换维度顺序，不然代码行为会非常怪异）


In [28]:
import torch

a = torch.randn(3, 2, 2)

index_0 = torch.tensor([False, False, True])
index_1 = torch.tensor([True, False])
index_2 = torch.tensor([[False, True], [True, False], [True, True]])
print(a)
# 比较有用的用法
print(a[index_0])
print(a[:, index_1, 1])
print(a[index_2])

# 先在 dim=1 上 索引位置 1，然后剩下 dim=[0,2] 匹配 index_2
print(a[index_2, 1])

# 没有找到一种一次索引写法可以先索引 dim=2 处位置 1，然后再将剩下的 dim=[0,1] 匹配 index2
# 不过可以写成以下方式
print(a[index_2][:, 1])  # 或者 print(a[:, :, 1][index_2])
# 没意义，这代码非常令人迷惑。

tensor([[[-0.5820, -2.7869],
         [ 0.4172, -0.2438]],

        [[-0.8403,  0.3157],
         [-0.3307, -0.4388]],

        [[-0.3768, -1.1343],
         [-1.3863,  0.1587]]])
tensor([[[-0.3768, -1.1343],
         [-1.3863,  0.1587]]])
tensor([[-2.7869],
        [ 0.3157],
        [-1.1343]])
tensor([[ 0.4172, -0.2438],
        [-0.8403,  0.3157],
        [-0.3768, -1.1343],
        [-1.3863,  0.1587]])
tensor([-0.2438, -0.3307, -1.3863,  0.1587])
tensor([-0.2438,  0.3157, -1.1343,  0.1587])


## fancy indexing

花式索引，使用一组 *long tensor* 进行索引。

条件：组内的 *long tensor* 能够广播至同一形状，每个元素所在的 *long tensor* 想要索引的维度不越界。

只考虑 *common indexing* 和 *fancy indexing* 的情况，其作用机制是先处理 *common indexing*，将保留维度看作整体元素，接着处理 *fancy indexing* 的维度，首先将组内向量广播至同一形状，然后将其每个对应位置的值组成一组坐标，按照最终广播形状索引 n 组坐标对应的位置。

In [32]:
a = torch.tensor([0, 1 ,2, 3, 4])
idx_0 = torch.tensor([[3, 2],[1, 4]])
print(a[idx_0])
b = torch.tensor([[0, 1], [2, 3], [4, 5]])
idx_0 = torch.tensor([[1, 0],[2, 1]])
idx_1 = torch.tensor([0, 1])
print(b[idx_0, idx_1])

tensor([[3, 2],
        [1, 4]])
tensor([[2, 1],
        [4, 3]])


In [72]:
# 有一个非常有意思的操作
def arrange_at_dim(arr_size, at_dim, total_dim):
    data = torch.arange(arr_size)
    assert at_dim < total_dim, "expect at_dim < total_dim"
    prefix = [1] * at_dim
    suffix = [1] * (total_dim - 1 - at_dim)
    return data.view(*prefix, -1, *suffix).contiguous()

a = torch.arange(2 * 3 * 4).view(2, 3, 4).contiguous()
# 利用这组特殊的tensor进行fancy indexing，被索引张量每一个位置都在原始位置被索引到。
index_0 = arrange_at_dim(a.shape[0], 0, a.ndim)
index_1 = arrange_at_dim(a.shape[1], 1, a.ndim)
index_2 = arrange_at_dim(a.shape[2], 2, a.ndim)
print(a[index_0, index_1, index_2] == a)
# 原因是这组tensor在广播之后恰好构成了与原始张量形状相同，并且每组坐标正好索引到当前位置
print(index_0, index_1, index_2, sep="\n")
print(*torch.broadcast_tensors(index_0, index_1, index_2), sep="\n")

tensor([[[True, True, True, True],
         [True, True, True, True],
         [True, True, True, True]],

        [[True, True, True, True],
         [True, True, True, True],
         [True, True, True, True]]])
tensor([[[0]],

        [[1]]])
tensor([[[0],
         [1],
         [2]]])
tensor([[[0, 1, 2, 3]]])
tensor([[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]]])
tensor([[[0, 0, 0, 0],
         [1, 1, 1, 1],
         [2, 2, 2, 2]],

        [[0, 0, 0, 0],
         [1, 1, 1, 1],
         [2, 2, 2, 2]]])
tensor([[[0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3]],

        [[0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3]]])


In [85]:
# 看看 torch.gather 和 torch.scatter 的作用机制
# torch.gather(input, dim, index)
# out[i][j][k] = input[index[i][j][k]][j][k]  # if dim == 0
# out[i][j][k] = input[i][index[i][j][k]][k]  # if dim == 1
# out[i][j][k] = input[i][j][index[i][j][k]]  # if dim == 2

# scatter(input, dim, index, src)
# input[index[i][j][k]][j][k] = src[i][j][k]  # if dim == 0
# input[i][index[i][j][k]][k] = src[i][j][k]  # if dim == 1
# input[i][j][index[i][j][k]] = src[i][j][k]  # if dim == 2

# 可以认为是将上面的 特殊组合 + 替换指定的 dim 为 index 的 fancy indexing

def my_gather(input_t, dim, index):
    all_idx = []
    for i in range(index.ndim):
        all_idx.append(arrange_at_dim(index.shape[i], i, input_t.ndim) if i != dim else index)
        
    return input_t[all_idx]
 
rst = torch.gather(torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]]), dim=0, index=torch.tensor([[1], [0], [2]]))
print(rst)
print(my_gather(torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]]), dim=0, index=torch.tensor([[1], [0], [2]])))

rst = torch.gather(torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]]), dim=1, index=torch.tensor([[1], [0], [2]]))
print(rst)
print(my_gather(torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]]), dim=1, index=torch.tensor([[1], [0], [2]])))


tensor([[3],
        [0],
        [6]])
tensor([[3],
        [0],
        [6]])
tensor([[1],
        [3],
        [8]])
tensor([[1],
        [3],
        [8]])
